# Ponderada de Cadeias de Markov (AS-IS vs TO-BE)
Nesta ponderada, aplico diretamente o que aprendi e entendi do tutorial do DataCamp disponibilizado no Card da Adalove (https://www.datacamp.com/tutorial/markov-chains-python-tutorial) à proposta de solução
A abordagem (seguindo  o tutorial da DataCamp) será usar uma matriz 3×3,  fazer o sorteio do próximo estado com `np.random.choice` usando a linha do estado atual, repetir as simulações e contar as frequências.


## 1) Contexto e premissas simples
Mantive **3 estados** (como no exemplo do tutorial) para comparar o fluxo **AS-IS** (atual) e o **TO-BE** (que estou propondo, como grupo 2, Estação Brigadeiro):

- `Editar` → trabalho em andamento
- `Revisar` → validação/checagem
- `Publicar` → documento aceito (sucesso)

## 2) Imports

In [37]:

import numpy as np
rng = np.random.default_rng(7)


## 3) Estados e matrizes (3×3, linhas somam 1)
Estados: `[Editar, Revisar, Publicar]`

- **AS-IS**: mais loops em `Revisar`, mais retorno de `Publicar` para `Revisar` (retrabalho).
- **TO-BE**: checagens automáticas + rastreabilidade + busca rápida → mais `Revisar → Publicar` e menos `Publicar → Revisar`.


In [38]:

states = ['Editar', 'Revisar', 'Publicar']

# --- AS-IS (hipótese didática) ---
P_as_is = np.array([
    [0.55, 0.40, 0.05],  # de Editar
    [0.20, 0.65, 0.15],  # de Revisar: fica muito em revisão e publica pouco
    [0.00, 0.15, 0.85],  # de Publicar: às vezes volta pra revisão (ajustes)
])

# --- TO-BE (hipótese didática orientada a automação) ---
P_to_be = np.array([
    [0.35, 0.60, 0.05],  # de Editar → vai mais para revisão estruturada
    [0.10, 0.30, 0.60],  # de Revisar: publica mais (checks + RAG)
    [0.00, 0.03, 0.97],  # de Publicar: quase não volta (qualidade/ci)
])

assert np.allclose(P_as_is.sum(axis=1), 1.0)
assert np.allclose(P_to_be.sum(axis=1), 1.0)


## 4) Simulação simples (conforme o tutorial)
- A cada passo, escolhemos o **próximo estado** usando `np.random.choice` com as probabilidades da **linha** do estado atual.
- Repetimos várias vezes e contamos em qual estado **terminamos**.


In [39]:

def simulate_once(P, start_state='Editar', steps=8):
    seq = [start_state]
    current = states.index(start_state)
    for _ in range(steps):
        probs = P[current]
        nxt = rng.choice(len(states), p=probs)
        seq.append(states[nxt])
        current = nxt
    return seq

def repeat_and_count(P, start_state='Editar', steps=8, runs=5000):
    last = {s: 0 for s in states}
    for _ in range(runs):
        seq = simulate_once(P, start_state, steps)
        last[seq[-1]] += 1
    total = float(runs)
    return {k: v/total for k, v in last.items()}


## 5) Experimentos (AS-IS vs TO-BE)
- **steps**: 10 (poucos passos, didático)
- **runs**: 5000 (frequências mais estáveis)


In [40]:

steps = 10
runs = 5000

# exemplo de uma trajetória
print('Trajetória (AS-IS):', ' -> '.join(simulate_once(P_as_is, steps=steps)))
print('Trajetória (TO-BE):', ' -> '.join(simulate_once(P_to_be, steps=steps)))

freq_as_is = repeat_and_count(P_as_is, steps=steps, runs=runs)
freq_to_be = repeat_and_count(P_to_be, steps=steps, runs=runs)

print('\nProbabilidade de terminar em cada estado (AS-IS):')
for k,v in freq_as_is.items():
    print(f'  {k}: {v:.3f}')

print('\nProbabilidade de terminar em cada estado (TO-BE):')
for k,v in freq_to_be.items():
    print(f'  {k}: {v:.3f}')


Trajetória (AS-IS): Editar -> Revisar -> Publicar -> Publicar -> Publicar -> Publicar -> Publicar -> Revisar -> Revisar -> Revisar -> Revisar
Trajetória (TO-BE): Editar -> Editar -> Editar -> Editar -> Revisar -> Publicar -> Publicar -> Publicar -> Publicar -> Publicar -> Publicar

Probabilidade de terminar em cada estado (AS-IS):
  Editar: 0.188
  Revisar: 0.403
  Publicar: 0.409

Probabilidade de terminar em cada estado (TO-BE):
  Editar: 0.007
  Revisar: 0.051
  Publicar: 0.942


## 6) Leitura rápida
**Métrica principal (didática):** probabilidade de terminar em `Publicar` após 10 passos.


In [41]:
pub_as_is = freq_as_is['Publicar']
pub_to_be  = freq_to_be['Publicar']
delta = (pub_to_be - pub_as_is) * 100
print(f"Publicar (AS-IS): {pub_as_is:.3f}")
print(f"Publicar (TO-BE): {pub_to_be:.3f}")
print(f"Variação (TO-BE - AS-IS): {delta:.1f}% maior")


Publicar (AS-IS): 0.409
Publicar (TO-BE): 0.942
Variação (TO-BE - AS-IS): 53.4% maior
